In [2]:
# --- Repo + path setup ---

import os, sys

# 1) Clone repo if it doesn't exist
if not os.path.exists('/content/bdh_challenge_2025'):
    %cd /content
    !git clone https://github.com/arionandrei2000/bdh_challenge_2025.git

# 2) Move into repo root
%cd /content/bdh_challenge_2025
print("PWD:", os.getcwd())
print("Repo contents:", os.listdir())

# 3) src is under notebooks/src, so add that parent folder to sys.path
src_parent = "/content/bdh_challenge_2025/notebooks"
if src_parent not in sys.path:
    sys.path.append(src_parent)

print("sys.path updated; ready to import src.")


/content
Cloning into 'bdh_challenge_2025'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 101 (delta 16), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 40.98 KiB | 5.12 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/bdh_challenge_2025
PWD: /content/bdh_challenge_2025
Repo contents: ['README.md', '.git', 'notebooks']
sys.path updated; ready to import src.


In [ ]:
# 01) Dependencies

# Core Python packages used in this notebook
# - pandas / numpy: data manipulation
# - pyarrow: for saving Parquet
# - requests: HTTP requests to GDC API
# - tqdm: progress bars
!pip install pandas numpy pyarrow requests tqdm pyranges --quiet

# Download GDC client
!wget https://gdc.cancer.gov/files/public/file/gdc-client_v1.6.1_Ubuntu_x64.zip -O gdc.zip -q
!unzip -o gdc.zip > /dev/null
!chmod +x gdc-client

print("Installed Python deps and downloaded gdc-client.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.0 MB/s eta 0:00:00
  Installing build dependencies ... canceledERROR: Operation cancelled by user
^C


KeyboardInterrupt: 

In [3]:

# 02) Mount GDrive
# This lets us store large TCGA data in Drive so it persists across sessions

from google.colab import drive
drive.mount('/content/drive')

print(" Google Drive mounted at /content/drive")


Mounted at /content/drive
 Google Drive mounted at /content/drive


In [4]:
# 03) Project Paths in Google Drive

from pathlib import Path
import os

# Main project root on Google Drive
PROJECT_ROOT = Path("/content/drive/MyDrive/bdh_challenge_2025_data")
PROJECT_ROOT.mkdir(exist_ok=True)

# Data directories

# Generic data directory
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# Directory where raw TCGA STAR count files will be stored
RNA_DIR = PROJECT_ROOT / "tcga_rna"
RNA_DIR.mkdir(exist_ok=True)

# Directory where processed matrices, tokenized data, embeddings
PROCESSED_DIR = PROJECT_ROOT / "processed"
PROCESSED_DIR.mkdir(exist_ok=True)

print("Project root :", PROJECT_ROOT)
print("DATA_DIR     :", DATA_DIR)
print("RNA_DIR      :", RNA_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)




Project root : /content/drive/MyDrive/bdh_challenge_2025_data
DATA_DIR     : /content/drive/MyDrive/bdh_challenge_2025_data/data
RNA_DIR      : /content/drive/MyDrive/bdh_challenge_2025_data/tcga_rna
PROCESSED_DIR: /content/drive/MyDrive/bdh_challenge_2025_data/processed


In [5]:
# 04) Imports and TCGA cohort

import os
import json
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

# We explicitly list the TCGA projects we will include. THIS CAN BE CHANGED!

BULK_RNABERT_PROJECTS = [
    "TCGA-BRCA",  # Breast invasive carcinoma
    "TCGA-BLCA",  # Bladder urothelial carcinoma
    "TCGA-GBM",   # Glioblastoma multiforme
    "TCGA-LGG",   # Lower grade glioma
    "TCGA-LUAD",  # Lung adenocarcinoma
    "TCGA-UCEC",  # Uterine corpus endometrial carcinoma
]

print("Cohorts for this run:")
for p in BULK_RNABERT_PROJECTS:
    print(" -", p)


Cohorts for this run:
 - TCGA-BRCA
 - TCGA-BLCA
 - TCGA-GBM
 - TCGA-LGG
 - TCGA-LUAD
 - TCGA-UCEC


In [ ]:

# 05) QUERY GDC FOR GENE EXPRESSION (STAR - COUNTS)

# We ask GDC:
#  - data_category: Transcriptome Profiling
#  - data_type: Gene Expression Quantification
#  - projects: the 6 TCGA projects above


files_query = {
    "filters": {
        "op": "and",
        "content": [
            {
                "op": "in",
                "content": {
                    "field": "cases.project.project_id",
                    "value": BULK_RNABERT_PROJECTS,
                },
            },
            {
                "op": "in",
                "content": {
                    "field": "data_category",
                    "value": ["Transcriptome Profiling"],
                },
            },
            {
                "op": "in",
                "content": {
                    "field": "data_type",
                    "value": ["Gene Expression Quantification"],
                },
            },
        ],
    },
    "format": "JSON",
    "size": 20000,
    "fields": (
        "file_id,file_name,"
        "cases.submitter_id,cases.project.project_id,"
        "data_category,data_type,analysis.workflow_type"
    ),
}

resp = requests.post(
    "https://api.gdc.cancer.gov/files",
    headers={"Content-Type": "application/json"},
    data=json.dumps(files_query),
)

resp_json = resp.json()
files_json = resp_json.get("data", {}).get("hits", [])

print("HTTP status:", resp.status_code)
print("Number of files for selected projects:", len(files_json))

if len(files_json) == 0:
    print(" GDC returned 0 files. Pagination/debug:")
    print(json.dumps(resp_json.get("data", {}).get("pagination", {}), indent=2))
    raise RuntimeError("GDC returned 0 files for this query. Adjust filters or check connection.")

# Convert JSON list to DataFrame
files_df = pd.json_normalize(files_json)

def _get_first_case(x):
    """Helper: for 'cases' field that is a list, return the first dict or {}."""
    if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict):
        return x[0]
    return {}

# Extract submitter_id and project_id from nested 'cases' field
files_df["submitter_id"] = files_df["cases"].apply(
    lambda x: _get_first_case(x).get("submitter_id", None)
)
files_df["project_id"] = files_df["cases"].apply(
    lambda x: _get_first_case(x).get("project_id", None)
)

print("Unique workflows in these files:")
print(files_df["analysis.workflow_type"].value_counts())

files_df[
    ["file_id", "file_name", "submitter_id", "project_id",
     "data_category", "data_type", "analysis.workflow_type"]
].head()


HTTP status: 200
Number of files for selected projects: 3777
Unique workflows in these files:
analysis.workflow_type
STAR - Counts    3777
Name: count, dtype: int64


,file_id,file_name,submitter_id,project_id,data_category,data_type,analysis.workflow_type
0,9dc09c86-c728-4bd9-b2b6-2d9962dad662,d1f1743c-5fd9-4ae8-90c2-8c3e2d475d1b.rna_seq.a...,TCGA-EW-A2FS,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts
1,95668f0b-130d-44d4-94c0-ba7a4e7798e6,6365a756-2e65-42cb-be4f-1f726915ca94.rna_seq.a...,TCGA-OL-A6VR,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts
2,461fda5d-d6e6-4354-b035-c302cc43b03f,30285113-d411-475a-a8ed-1fb66be72f28.rna_seq.a...,TCGA-E9-A226,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts
3,30ff778c-844b-4140-9025-7ab1938f10a9,5167da8c-2b1c-4139-a2c4-355e9f07d0be.rna_seq.a...,TCGA-A8-A08H,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts
4,427a04c9-9b48-49de-8a47-2adc4e1dd32a,fead73ce-2e66-4647-8b6c-b8bbdaaf30fe.rna_seq.a...,TCGA-D8-A27H,None,Transcriptome Profiling,Gene Expression Quantification,STAR - Counts


In [ ]:
# 06) Create MANIFEST & DOWNLOAD star counts TO GDrive

# GDC client expects a manifest file with an "id" column listing file_ids.

manifest = files_df[["file_id"]].rename(columns={"file_id": "id"})
manifest_path = PROJECT_ROOT / "manifest_star_counts.txt"
manifest.to_csv(manifest_path, sep="\t", index=False)

print("Manifest written to:", manifest_path)
print("Total files listed:", manifest.shape[0])

# Download all files into RNA_DIR (in Google Drive).
# gdc-client will skip files that are already complete.
!./gdc-client download -m {manifest_path} -d {RNA_DIR}

print(" Download finished (check tcga_rna/ in Google Drive).")


Manifest written to: /content/drive/MyDrive/bdh_challenge_2025_data/manifest_star_counts.txt
Total files listed: 3777
/bin/bash: line 1: ./gdc-client: No such file or directory
 Download finished (check tcga_rna/ in Google Drive).


In [ ]:
# 07) Getting the STAR counts matrix
#
# This version:
#  - Only processes STAR gene count files (augmented_star_gene_counts.tsv)
#  - Reads with header row
#  - Drops N_* and __* summary rows
#  - Works with partial downloads
#  - Stores counts as float32 (half the RAM vs float64)

id_to_sample = dict(zip(files_df["file_id"], files_df["submitter_id"]))
print("Number of file_id → sample mappings:", len(id_to_sample))

all_files = []
for root, dirs, files in os.walk(RNA_DIR):
    for f in files:
        # STAR gene count files usually contain 'rna_seq' and 'gene_counts'
        if f.endswith(".tsv") and "rna_seq" in f and "gene_counts" in f:
            all_files.append(os.path.join(root, f))

print("Number of STAR gene count files on disk:", len(all_files))

MAX_FILES = None
if MAX_FILES is not None:
    all_files = all_files[:MAX_FILES]
    print(f"Using only first {len(all_files)} files.")

gene_index = None
matrix = {}
skipped = []

for fpath in tqdm(all_files):
    fname = os.path.basename(fpath)

    # Folder name created by gdc-client is the file_id
    file_id = os.path.basename(os.path.dirname(fpath))
    sample = id_to_sample.get(file_id)
    if sample is None:
        skipped.append((fname, "no sample mapping"))
        continue

    # Read gene counts table; ignore comment lines starting with '#'
    df = pd.read_csv(fpath, sep="\t", comment="#")

    # Identify gene ID column
    if "gene_id" in df.columns:
        gene_col = "gene_id"
    elif "Geneid" in df.columns:
        gene_col = "Geneid"
    else:
        skipped.append((fname, f"no gene_id column, cols = {df.columns.tolist()}"))
        continue

    # Identify counts column
    if "unstranded" in df.columns:
        counts_col = "unstranded"
    else:
        # pick the first non-gene_id/gene_name column as fallback
        candidate_cols = [c for c in df.columns if c not in [gene_col, "gene_name"]]
        if not candidate_cols:
            skipped.append((fname, "no suitable count column"))
            continue
        counts_col = candidate_cols[0]

    # Drop technical summary rows: N_unmapped, N_multimapping, __no_feature, etc.
    df = df[~df[gene_col].astype(str).str.startswith("N_")]
    df = df[~df[gene_col].astype(str).str.startswith("__")]
    df = df.reset_index(drop=True)

    # Initialize or check gene ordering
    if gene_index is None:
        gene_index = df[gene_col].values
    else:
        if not np.array_equal(gene_index, df[gene_col].values):
            raise ValueError(f"Gene order mismatch in file: {fpath}")

    # Extract counts as float32 (saves RAM vs float64)
    counts = pd.to_numeric(df[counts_col], errors="raise").astype("float32").values
    matrix[sample] = counts

# Build counts matrix (genes x samples) and store as float32
expr_counts = pd.DataFrame(matrix, index=gene_index).astype("float32")

print("Counts matrix shape (genes x samples):", expr_counts.shape)
display(expr_counts.iloc[:5, :5])

print("Skipped files:", len(skipped))
if skipped:
    print(skipped[:5])


Number of file_id → sample mappings: 3777
Number of STAR gene count files on disk: 3777


100%|██████████| 3777/3777 [52:40<00:00,  1.20it/s]


Counts matrix shape (genes x samples): (60660, 3385)


,TCGA-P5-A5EY,TCGA-AR-A1AW,TCGA-E9-A1N5,TCGA-97-7941,TCGA-93-7347
ENSG00000000003.15,2377.0,2273.0,2036.0,11528.0,1381.0
ENSG00000000005.6,8.0,13.0,39.0,13.0,5.0
ENSG00000000419.13,649.0,2077.0,703.0,1547.0,998.0
ENSG00000000457.14,331.0,1459.0,764.0,1257.0,715.0
ENSG00000000460.17,79.0,726.0,251.0,232.0,195.0


Skipped files: 0


In [ ]:
# 08) Doing TPM-like normalization on Counts
# We approximatd TPM by:
#   TPM_like = (counts / sum(counts)) * 1e6 per sample.
# This keeps per-sample library size comparable across samples.

# Very close to using FPKM→TPM for a lot of downstream modeling use cases. LET'S double check this! Why?

def counts_to_tpm_like(column: pd.Series) -> pd.Series:
    total = column.sum()
    if total == 0:
        return column
    return (column / total) * 1e6

expr_tpm = expr_counts.apply(counts_to_tpm_like, axis=0)
print("TPM-like matrix shape (genes x samples):", expr_tpm.shape)
expr_tpm.iloc[:5, :5]


TPM-like matrix shape (genes x samples): (60660, 3385)


,TCGA-P5-A5EY,TCGA-AR-A1AW,TCGA-E9-A1N5,TCGA-97-7941,TCGA-93-7347
ENSG00000000003.15,47.031635,39.358894,69.550377,178.855652,23.861710
ENSG00000000005.6,0.158289,0.225106,1.332252,0.201694,0.086393
ENSG00000000419.13,12.841199,35.964989,24.014692,24.001535,17.244017
ENSG00000000457.14,6.549210,25.263803,26.098471,19.502218,12.354180
ENSG00000000460.17,1.563104,12.571296,8.574236,3.599455,3.369322


In [ ]:
# 09) Clean genes, reorient to samples x genes, log-transform, and save
#
# Starting point:
#   - expr_counts : (genes x samples), float32
#   - expr_tpm    : (genes x samples), TPM-like
#
# This cell:
#   (1) Strips Ensembl version suffixes (ENSG...15 -> ENSG...)
#   (2) Removes duplicates in BOTH matrices
#   (3) Intersects gene sets so counts & TPM have identical genes
#   (4) Reorients to (samples x genes)
#   (5) Applies log10(1 + TPM-like)
#   (6) Saves outputs

import gc
import numpy as np
import os

# 1) Strip Ensembl version suffix from gene IDs
expr_tpm.index    = expr_tpm.index.to_series().str.split(".").str[0]
expr_counts.index = expr_counts.index.to_series().str.split(".").str[0]


# 2) Remove duplicate genes IN BOTH MATRICES
expr_tpm    = expr_tpm[~expr_tpm.index.duplicated(keep="first")]
expr_counts = expr_counts[~expr_counts.index.duplicated(keep="first")]

# 3) Intersect genes so both matrices have identical gene sets
common_genes = expr_tpm.index.intersection(expr_counts.index)
expr_tpm    = expr_tpm.loc[common_genes].sort_index()
expr_counts = expr_counts.loc[common_genes].sort_index()

print("After strip + dedupe + intersect:")
print("  expr_counts shape:", expr_counts.shape)
print("  expr_tpm    shape:", expr_tpm.shape)

# 4) Reorient to (samples x genes)
expr_counts_sxg = expr_counts.T.astype("float32").copy()
expr_tpm_sxg    = expr_tpm.T.astype("float32").copy()

expr_counts_sxg.index.name = "sample_id"
expr_tpm_sxg.index.name    = "sample_id"

print("Counts   (samples x genes):", expr_counts_sxg.shape)
print("TPM-like (samples x genes):", expr_tpm_sxg.shape)

gc.collect()

# 5) Log-transform: log10(1 + TPM-like)
expr_tpm_log = np.log10(1.0 + expr_tpm_sxg.astype("float32"))
print("Log10(1 + TPM-like) shape:", expr_tpm_log.shape)

# 6) Save matrices to disk (into Google Drive)
from pathlib import Path

# Use the PROCESSED_DIR defined in the paths cell
OUT_DIR = PROCESSED_DIR  # this is a Path object pointing to .../bdh_challenge_2025_data/processed
OUT_DIR.mkdir(parents=True, exist_ok=True)

expr_counts_sxg.to_parquet(OUT_DIR / "tcga_star_counts_sxg.parquet")
expr_tpm_sxg.to_parquet(OUT_DIR / "tcga_tpm_like_sxg.parquet")
expr_tpm_log.to_parquet(OUT_DIR / "tcga_tpm_like_log10_sxg.parquet")

print("Saved processed matrices to:", OUT_DIR)

# Optional: verify files
print("Processed directory contents:")
for p in OUT_DIR.iterdir():
    print("  -", p.name)


After strip + dedupe + intersect:
  expr_counts shape: (60616, 3385)
  expr_tpm    shape: (60616, 3385)
Counts   (samples x genes): (3385, 60616)
TPM-like (samples x genes): (3385, 60616)
Log10(1 + TPM-like) shape: (3385, 60616)
Saved processed matrices to: /content/drive/MyDrive/bdh_challenge_2025_data/processed
Processed directory contents:
  - tcga_star_counts_sxg.parquet
  - tcga_tpm_like_sxg.parquet
  - tcga_tpm_like_log10_sxg.parquet


In [ ]:
# 10) Align our TCGA log-TPM matrix to BulkRNABert gene list (common_gene_id.txt)

import pandas as pd
import numpy as np
from pathlib import Path

GENE_LIST_PATH = DATA_DIR / "common_gene_id.txt"
TPM_LOG_PATH   = PROCESSED_DIR / "tcga_tpm_like_log10_sxg.parquet"

print("Gene list path:", GENE_LIST_PATH)
print("Log TPM matrix path:", TPM_LOG_PATH)

# 1) Load authors' gene list
# File is one gene ID per line
with open(GENE_LIST_PATH) as f:
    author_genes = [line.strip() for line in f if line.strip()]

author_genes = pd.Index(author_genes, name="gene_id")
print("Genes in authors' list:", len(author_genes))
print(author_genes[:5])

# 2) Load our processed log TPM matrix (samples x genes)
expr_tpm_log = pd.read_parquet(TPM_LOG_PATH)
print("Our TCGA log-TPM matrix shape (samples x genes):", expr_tpm_log.shape)

# Make sure columns are strings (gene IDs)
expr_tpm_log.columns = expr_tpm_log.columns.astype(str)

# 3) Check overlaps between our genes and authors' genes
our_genes = pd.Index(expr_tpm_log.columns, name="gene_id")
common = author_genes.intersection(our_genes)
missing_in_ours = author_genes.difference(our_genes)
extra_in_ours = our_genes.difference(author_genes)

print("\n=== Gene set diagnostics ===")
print("Genes in our matrix        :", len(our_genes))
print("Genes in authors' list     :", len(author_genes))
print("Common genes               :", len(common))
print("Authors' genes missing in ours:", len(missing_in_ours))
print("Our genes not in authors' list:", len(extra_in_ours))

if len(common) == 0:
    raise ValueError("No overlap between TCGA genes and author gene list – check ID format!")

print("\nExample authors' genes missing in our matrix:", list(missing_in_ours[:10]))

# 4) Reindex to authors' order (BulkRNABert canonical order)
# This will create columns for ALL author_genes, preserving order.
# Genes that we don't have will be NaN.
expr_aligned = expr_tpm_log.reindex(columns=author_genes)

print("\nAligned matrix shape (samples x authors' genes):", expr_aligned.shape)

# 5) Handle missing values & basic sanity checks
nan_count = expr_aligned.isna().sum().sum()
print("Total NaN values in aligned matrix:", nan_count)

# Strategy: fill missing genes with 0.0 (log10(1+TPM)=0 => TPM=0)
expr_aligned = expr_aligned.fillna(0.0).astype("float32")

# Quick distribution check
print("\nAligned matrix summary (first 5 genes):")
print(expr_aligned.iloc[:, :5].describe().T)

# 6) Save aligned matrix
ALIGNED_PATH = PROCESSED_DIR / "tcga_tpm_like_log10_bulkrnabert_aligned.parquet"
expr_aligned.to_parquet(ALIGNED_PATH)

print("\nSaved aligned matrix to:", ALIGNED_PATH)


Gene list path: /content/drive/MyDrive/bdh_challenge_2025_data/data/common_gene_id.txt
Log TPM matrix path: /content/drive/MyDrive/bdh_challenge_2025_data/processed/tcga_tpm_like_log10_sxg.parquet
Genes in authors' list: 19062
Index(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419',
       'ENSG00000000457', 'ENSG00000000460'],
      dtype='object', name='gene_id')
Our TCGA log-TPM matrix shape (samples x genes): (3385, 60616)

=== Gene set diagnostics ===
Genes in our matrix        : 60616
Genes in authors' list     : 19062
Common genes               : 19062
Authors' genes missing in ours: 0
Our genes not in authors' list: 41554

Example authors' genes missing in our matrix: []

Aligned matrix shape (samples x authors' genes): (3385, 19062)
Total NaN values in aligned matrix: 0

Aligned matrix summary (first 5 genes):
                  count      mean       std       min       25%       50%  \
gene_id                                                                     
ENSG0000

In [6]:
# 11) Download TCGA clinical data (cases endpoint) for the selected cohorts

import json
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

print("Projects:", BULK_RNABERT_PROJECTS)

cases_query = {
    "filters": {
        "op": "in",
        "content": {
            "field": "project.project_id",
            "value": BULK_RNABERT_PROJECTS,
        },
    },
    "format": "JSON",
    "size": 10000,  # enough for all cases in these 6 cohorts
    "fields": ",".join([
        "case_id",
        "submitter_id",
        "project.project_id",
        # demographics
        "demographic.gender",
        "demographic.race",
        "demographic.ethnicity",
        # diagnoses (may be multiple per case)
        "diagnoses.vital_status",
        "diagnoses.days_to_death",
        "diagnoses.days_to_last_follow_up",
        "diagnoses.days_to_last_known_alive",
        "diagnoses.diagnosis_is_primary_disease",
        "diagnoses.age_at_diagnosis",
        "diagnoses.primary_diagnosis",
        "diagnoses.tumor_stage",
        "diagnoses.morphology",
    ]),
}

resp = requests.post(
    "https://api.gdc.cancer.gov/cases",
    headers={"Content-Type": "application/json"},
    data=json.dumps(cases_query),
)

print("HTTP status:", resp.status_code)
cases_json = resp.json()

cases_hits = cases_json.get("data", {}).get("hits", [])
print("Number of case records returned:", len(cases_hits))

if len(cases_hits) == 0:
    raise RuntimeError("No clinical cases returned from GDC. Check query or network.")


Projects: ['TCGA-BRCA', 'TCGA-BLCA', 'TCGA-GBM', 'TCGA-LGG', 'TCGA-LUAD', 'TCGA-UCEC']
HTTP status: 200
Number of case records returned: 3788


In [7]:
def _safe_get(d, *keys, default=None):
    """Nested dict .get helper: _safe_get(d, 'a', 'b') -> d['a']['b'] if present."""
    cur = d
    for k in keys:
        if cur is None:
            return default
        if isinstance(cur, dict):
            cur = cur.get(k, default)
        else:
            return default
    return cur if cur is not None else default


def _pick_primary_diagnosis(diags):
    """
    diags: list of diagnosis dicts (may be empty).
    Preference:
      - any with diagnosis_is_primary_disease == True
      - else the first one
    """
    if not diags:
        return None
    for d in diags:
        if d.get("diagnosis_is_primary_disease") is True:
            return d
    return diags[0]


rows = []
for c in tqdm(cases_hits, desc="Building clinical rows"):
    submitter = c.get("submitter_id")    # e.g. TCGA-P5-A5EY
    if submitter is None:
        continue

    project_id = _safe_get(c, "project", "project_id")
    cohort = None
    if project_id is not None:
        # TCGA-BRCA -> BRCA
        cohort = project_id.replace("TCGA-", "")

    demo = c.get("demographic") or {}
    gender = demo.get("gender")
    race = demo.get("race")
    ethnicity = demo.get("ethnicity")

    diags = c.get("diagnoses") or []
    primary = _pick_primary_diagnosis(diags)

    vital_status = None
    days_to_death = None
    days_to_followup = None
    age_at_diag = None
    tumor_stage = None
    primary_dx = None
    morphology = None

    if primary is not None:
        vital_status = primary.get("vital_status")
        days_to_death = primary.get("days_to_death")
        days_to_followup = (
            primary.get("days_to_last_follow_up")
            if primary.get("days_to_last_follow_up") is not None
            else primary.get("days_to_last_known_alive")
        )
        age_at_diag = primary.get("age_at_diagnosis")
        tumor_stage = primary.get("tumor_stage")
        primary_dx = primary.get("primary_diagnosis")
        morphology = primary.get("morphology")

    # Compute survival_time and event
    survival_time = None
    event = None

    # Use days_to_death if dead, else days_to_last_follow_up/known_alive
    if vital_status is not None:
        if vital_status.upper() == "DEAD" and days_to_death is not None:
            event = 1
            survival_time = days_to_death
        elif days_to_followup is not None:
            event = 0
            survival_time = days_to_followup

    row = {
        "sample_id": submitter,
        "project_id": project_id,
        "cohort": cohort,
        "gender": gender,
        "race": race,
        "ethnicity": ethnicity,
        "vital_status": vital_status,
        "days_to_death": days_to_death,
        "days_to_last_followup": days_to_followup,
        "age_at_diagnosis": age_at_diag,
        "tumor_stage": tumor_stage,
        "primary_diagnosis": primary_dx,
        "morphology": morphology,
        "survival_time": survival_time,
        "event": event,
    }
    rows.append(row)

clinical_df = pd.DataFrame(rows)
print("Raw clinical table shape:", clinical_df.shape)
clinical_df.head()


Building clinical rows: 100%|██████████| 3788/3788 [00:00<00:00, 193624.15it/s]

Raw clinical table shape: (3788, 15)


,sample_id,project_id,cohort,gender,race,ethnicity,vital_status,days_to_death,days_to_last_followup,age_at_diagnosis,tumor_stage,primary_diagnosis,morphology,survival_time,event
0,TCGA-E2-A107,TCGA-BRCA,BRCA,female,white,not hispanic or latino,None,None,1047.0,20065.0,None,"Lobular carcinoma, NOS",8520/3,None,None
1,TCGA-AC-A62X,TCGA-BRCA,BRCA,female,black or african american,not hispanic or latino,None,None,417.0,26627.0,None,"Infiltrating duct carcinoma, NOS",8500/3,None,None
2,TCGA-E2-A156,TCGA-BRCA,BRCA,female,white,not hispanic or latino,None,None,726.0,22422.0,None,"Infiltrating duct carcinoma, NOS",8500/3,None,None
3,TCGA-A7-A56D,TCGA-BRCA,BRCA,female,black or african american,not hispanic or latino,None,None,448.0,30783.0,None,"Infiltrating duct carcinoma, NOS",8500/3,None,None
4,TCGA-AR-A0U1,TCGA-BRCA,BRCA,female,white,not hispanic or latino,None,None,4052.0,13467.0,None,"Infiltrating duct carcinoma, NOS",8500/3,None,None


In [8]:
# 13) Align clinical table to expression samples and save to PROCESSED_DIR

# Load your main log-TPM aligned matrix (samples x genes)
EXPR_ALIGNED_PATH = PROCESSED_DIR / "tcga_tpm_like_log10_bulkrnabert_aligned.parquet"
expr_aligned = pd.read_parquet(EXPR_ALIGNED_PATH)
print("Expression aligned shape:", expr_aligned.shape)

# Align by sample_id
clinical_df = clinical_df.set_index("sample_id")

common_samples = expr_aligned.index.intersection(clinical_df.index)
print("Samples in expression:", expr_aligned.shape[0])
print("Samples in clinical  :", clinical_df.shape[0])
print("Common samples       :", len(common_samples))

clinical_aligned = clinical_df.loc[common_samples].copy()

# Basic diagnostics
print("\nNon-null survival_time:", clinical_aligned["survival_time"].notna().sum())
print("Non-null event        :", clinical_aligned["event"].notna().sum())
print("\nCohort counts:")
print(clinical_aligned["cohort"].value_counts())

# Save cleaned, aligned clinical table
CLIN_OUT = PROCESSED_DIR / "tcga_clinical_bulkrnabert_cohorts.parquet"
clinical_aligned.to_parquet(CLIN_OUT)

print("\nSaved aligned clinical to:", CLIN_OUT)


Expression aligned shape: (3385, 19062)
Samples in expression: 3385
Samples in clinical  : 3788
Common samples       : 3385

Non-null survival_time: 0
Non-null event        : 0

Cohort counts:
cohort
BRCA    1095
UCEC     557
LUAD     518
LGG      516
BLCA     406
GBM      293
Name: count, dtype: int64

Saved aligned clinical to: /content/drive/MyDrive/bdh_challenge_2025_data/processed/tcga_clinical_bulkrnabert_cohorts.parquet
